In [ ]:
import pandas as pd
import re

# Example DataFrame
df = pd.DataFrame({
    'my_column': ['A;B', 'B;C', 'A', 'C;D', 'B', 'M;N', 'O;P', 'A;M', 'a-b; m n', None],
    'customer_number': ['123.0', '456.0', '789', 'abc', '12 3', '45-6', '78%9', '?', None, '001.0']
})

# Split the column by ';' into lists
df['my_column_split'] = df['my_column'].str.split(';')

# Clean values: strip, capitalize, replace spaces and hyphens with underscores
def clean_list(lst):
    if not isinstance(lst, list):
        return []
    return [x.strip().upper().replace(' ', '_').replace('-', '_') for x in lst]

df['my_column_split'] = df['my_column_split'].apply(clean_list)

# Clean customer_number field
def clean_customer_number(val):
    if pd.isnull(val):
        return ''
    val = str(val).strip()
    if val.endswith('.0'):
        val = val[:-2]
    val = re.sub(r'[^A-Za-z0-9_]', '_', val)  # Replace unwanted chars with underscore
    return val

df['customer_number_clean'] = df['customer_number'].apply(clean_customer_number)

# Explode the lists into rows
exploded = df.explode('my_column_split')

# Create binary columns for each unique value
dummies = pd.get_dummies(exploded['my_column_split'])

# Group back to original rows by max (if any value is 1, keep 1)
binary_df = dummies.groupby(exploded.index).max()

# Create business and savings flags
business_flags = {'A', 'B', 'C', 'D', 'E', 'X', 'F'}
savings_flags = {'M', 'N', 'O', 'P'}

binary_df['business'] = binary_df[list(business_flags & set(binary_df.columns))].any(axis=1).astype(int)
binary_df['savings'] = binary_df[list(savings_flags & set(binary_df.columns))].any(axis=1).astype(int)

# Concatenate with original DataFrame if needed
result = pd.concat([df, binary_df], axis=1)

result